In [1]:
import random

from generate_fol_strings import main as generate_fol_strings
from itertools import combinations
from pyetr import View
from pyetr.inference import (
    default_inference_procedure,
    classically_valid_inference_procedure,
)

In [5]:
formulas = generate_fol_strings(vocab_size=4, depth=3)

Generated 820 Boolean formulas.


In [34]:
for p1, p2 in combinations(random.sample(formulas, 2), r=2):
    print(f"Formulas: {p1}; {p2}")
    v1 = View.from_fol(p1)
    v2 = View.from_fol(p2)
    print(f"View 1: {v1.to_str()}")
    print(f"View 2: {v2.to_str()}")

    print("ETR predicts: ", default_inference_procedure((v1, v2)))
    print("Classical predicts: ", classically_valid_inference_procedure((v1, v2)))



Formulas: (~A() ∧ C()); (A() ∨ D())
View 1: {C()~A()}
View 2: {D(),A()}
ETR predicts:  {D()}
Classical predicts:  {D()}


In [38]:
len(default_inference_procedure((v1, v2)).stage)

1

In [6]:
formulas = generate_fol_strings(vocab_size=4, depth=3)

Generated 820 Boolean formulas.


In [ ]:
cases = []

In [46]:
from pyetr import PredicateAtom
from pyetr.stateset import State
from pyetr.atoms import Atom

def statement_from_atom(atom: Atom, statement_mappings: dict) -> str:
    if atom.predicate.verifier:
        return statement_mappings[atom.predicate.name]["verifier"]
    else:
        return statement_mappings[atom.predicate.name]["falsifier"]

In [45]:
from pyetr.stateset import State


def statement_from_state(state: State, statement_mappings: dict) -> str:
    return " and ".join(
        [statement_from_atom(atom, statement_mappings) for atom in state.atoms]
    )

In [75]:
def view_stage_to_natural_language(view: View) -> str:
    statements = [
        {
            "verifier": "there is a king",
            "falsifier": "there isn't a king",
        },
        {
            "verifier": "there is a queen",
            "falsifier": "there isn't a queen",
        },
        {
            "verifier": "there is an ace",
            "falsifier": "there isn't an ace",
        },
        {
            "verifier": "there is a jack",
            "falsifier": "there isn't a jack,"
        }
    ]

    statement_mappings = {}
    i = 0
    for a in view.atoms:
        # If we've already mapped this predicate, continue
        if a.predicate.name in statement_mappings.keys(): continue

        # Make the predicate a verifier, assuming we'll see both verifier and falsifer
        if not a.predicate.verifier: a = ~a

        statement_mappings.update(
            {str(a.predicate.name): {
                "verifier": statements[i]["verifier"],
                "falsifier": statements[i]["falsifier"]
            }}
        )
        i += 1

    if len(view.stage) == 0: return ""

    if len(view.stage) == 1:
        statement = statement_from_state(list(view.stage)[0], statement_mappings)
    else:
        statement = "either " + ", or ".join(
            [statement_from_state(state, statement_mappings) for state in view.stage]
        )
    if statement == "": return ""
    return statement[0].upper() + statement[1:] + "."

In [66]:
v = View.from_str("{A()~B(),C()}")
v

{~B()A(),C()}

In [67]:
view_stage_to_natural_language(v)

"Either there isn't a king and there is an ace, or there is a queen."

In [68]:
list(v.stage)[0].atoms

{A(), ~B()}

In [69]:
type(list(v.atoms)[0].predicate.name)

str

In [70]:
list(v.atoms)[0].predicate.verifier

False

In [77]:
cases = []

for p1, p2 in combinations(random.sample(formulas, 4), r=2):
    v1 = View.from_fol(p1)
    v2 = View.from_fol(p2)

    etr_c = default_inference_procedure((v1, v2))
    classical_c = classically_valid_inference_procedure((v1, v2))

    if (
        len(etr_c.stage) == 1
        and len(classical_c.stage) != 1
        and not etr_c.is_verum
        and not etr_c.is_falsum
    ):
        print(f"Formulas: {p1}; {p2}")
        print(f"View 1: {v1.to_str()}")
        print(f"View 2: {v2.to_str()}")
        print("ETR predicts: ", etr_c)
        print("Classical predicts: ", classical_c)
        print()
    
    cases.append({
        "p1_formal": p1,
        "p2_formal": p2,
        "etr_c_formal": etr_c,
        "classical_c_formal": classical_c,
        "p1_nl": view_stage_to_natural_language(v1),
        "p2_nl": view_stage_to_natural_language(v2),
        "etr_c_nl": view_stage_to_natural_language(etr_c),
        "classical_c_nl": view_stage_to_natural_language(classical_c),
        "etr_categorical": len(etr_c.stage) == 1,
        "classical_categorical": len(classical_c.stage) == 1,
        "etr_classical_agreement": etr_c.stage == classical_c.stage,
    })

Formulas: ((A() ∨ ~A()) ∨ (A() ∨ ~C())); ((A() ∧ ~A()) ∨ (~A() ∧ ~A()))
View 1: {A(),~C(),~A()}
View 2: {~A()A(),~A()}
ETR predicts:  {~A()}
Classical predicts:  {~A()~C(),~A()}



In [78]:
cases

[{'p1_formal': '((A() ∨ ~A()) ∨ (A() ∨ ~C()))',
  'p2_formal': '(A() ∧ ~D())',
  'etr_c_formal': {0,~C()},
  'classical_c_formal': {~C(),0},
  'p1_nl': "Either there is a king, or there isn't a queen, or there isn't a king.",
  'p2_nl': "There isn't a king and there is a queen.",
  'etr_c_nl': "Either , or there isn't a king.",
  'classical_c_nl': "Either , or there isn't a king.",
  'etr_categorical': False,
  'classical_categorical': False,
  'etr_classical_agreement': True},
 {'p1_formal': '((A() ∨ ~A()) ∨ (A() ∨ ~C()))',
  'p2_formal': '((A() ∧ ~A()) ∨ (B() ∨ ~B()))',
  'etr_c_formal': {~A()B(),~A()~B(),~B()A(),B()A()},
  'classical_c_formal': {~C()B()A(),~A()~C()~B(),~B(),B(),~A()~C()B(),~C()~B()A()},
  'p1_nl': "Either there is a king, or there isn't a queen, or there isn't a king.",
  'p2_nl': "Either there isn't a queen, or there isn't a king and there is a king, or there is a queen.",
  'etr_c_nl': "Either there isn't a king and there is a queen, or there isn't a king and ther

In [86]:
cases[2]

{'p1_formal': '((A() ∨ ~A()) ∨ (A() ∨ ~C()))',
 'p2_formal': '((A() ∧ ~A()) ∨ (~A() ∧ ~A()))',
 'etr_c_formal': {~A()},
 'classical_c_formal': {~A()~C(),~A()},
 'p1_nl': "Either there is a king, or there isn't a queen, or there isn't a king.",
 'p2_nl': "Either there isn't a king and there is a king, or there isn't a king.",
 'etr_c_nl': "There isn't a king.",
 'classical_c_nl': "Either there isn't a king and there isn't a queen, or there isn't a king.",
 'etr_categorical': True,
 'classical_categorical': False,
 'etr_classical_agreement': False}